<a href="https://colab.research.google.com/github/alvinwalker314/DS-Unit-2-Regression-1/blob/master/Alvin_Kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install category_encoders

     |████████████████████████████████| 92kB 27.9MB/s 


In [0]:
import category_encoders as ce
from google.colab import files
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [0]:
sample_sub= pd.read_csv('https://raw.githubusercontent.com/alvinwalker314/DS-Unit-2-Classification-1/master/sample_submission.csv')
test_features= pd.read_csv('https://raw.githubusercontent.com/alvinwalker314/DS-Unit-2-Classification-1/master/test_features.csv')
train_features= pd.read_csv('https://raw.githubusercontent.com/alvinwalker314/DS-Unit-2-Classification-1/master/train_features.csv')
train_labels= pd.read_csv('https://raw.githubusercontent.com/alvinwalker314/DS-Unit-2-Classification-1/master/train_labels.csv')
pd.set_option('display.float_format', '{:.2f}'.format)

In [4]:
sample_sub.shape, test_features.shape, train_features.shape, train_labels.shape

((14358, 2), (14358, 40), (59400, 40), (59400, 2))

In [0]:
#Split training data
X_train= train_features
y_train= train_labels['status_group']

X_train, X_test, y_train, y_test= train_test_split(
    X_train, y_train, train_size=0.80, test_size=0.20,random_state= 20, stratify= y_train)


In [5]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((47520, 40), (11880, 40), (47520,), (11880,))

In [0]:
#numeric features
X_train_numeric= X_train.select_dtypes('number')
X_test_numeric= X_test.select_dtypes('number')
test_features_numeric= test_features.select_dtypes('number')

In [8]:
X_train_numeric.describe()

,id,amount_tsh,gps_height,longitude,latitude,num_private,region_code,district_code,population,construction_year
count,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00,47520.00
mean,37127.76,320.28,666.95,34.08,-5.71,0.49,15.31,5.64,177.70,1301.07
std,21433.61,3027.11,693.15,6.61,2.95,13.28,17.68,9.65,453.02,951.52
min,0.00,0.00,-90.00,0.00,-11.65,0.00,1.00,0.00,0.00,0.00
25%,18577.50,0.00,0.00,33.10,-8.55,0.00,5.00,2.00,0.00,0.00
50%,37094.50,0.00,366.00,34.92,-5.02,0.00,12.00,3.00,25.00,1986.00
75%,55592.25,21.25,1318.00,37.19,-3.33,0.00,17.00,5.00,210.00,2004.00
max,74242.00,350000.00,2770.00,40.35,-0.00,1776.00,99.00,80.00,15300.00,2013.00


In [0]:
X_train_numeric= X_train_numeric.drop(columns=['id', 'num_private'])
X_test_numeric= X_test_numeric.drop(columns=['id', 'num_private'])
test_features_numeric= test_features_numeric.drop(columns=['id', 'num_private'])

In [33]:
X_train_numeric.shape, X_test_numeric.shape, test_features_numeric.shape

((47520, 8), (11880, 8), (14358, 8))

In [0]:
#standarize features
scaler=StandardScaler()
X_train_scaled= scaler.fit_transform(X_train_numeric)
X_test_scaled= scaler.transform(X_test_numeric)

In [35]:
X_train_scaled.shape, X_test_scaled.shape

((47520, 8), (11880, 8))

In [9]:
#model with standarized numeric features
model= LogisticRegression(solver='lbfgs', multi_class= 'auto', max_iter=1000)
model.fit(X_train_scaled, y_train)
y_pred= model.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.5576599326599326

In [0]:
#coefs
functional_coef= pd.Series(model.coef_[0], X_train_numeric.columns)
non_functional_coef= pd.Series(model.coef_[1], X_train_numeric.columns)
needs_repairs_coef= pd.Series(model.coef_[2], X_train_numeric.columns)

In [0]:
#plotting coef for functional
fig= plt.figure(figsize=(10,10))
functional_coef.sort_values().plot.barh();
plt.xlabel('Effect percentage')
plt.ylabel('Features')
plt.title('Effects of different features on whether target is functional');

In [0]:
#plotting coef for not functional
fig= plt.figure(figsize=(10,10))
non_functional_coef.sort_values().plot.barh();
plt.xlabel('Effect percentage')
plt.ylabel('Features')
plt.title('Effects of different features on whether target is not functional');

In [0]:
#plotting coef for needs repairs
fig= plt.figure(figsize=(10,10))
needs_repairs_coef.sort_values().plot.barh();
plt.xlabel('Effect percentage')
plt.ylabel('Features')
plt.title('Effects of different features on whether target is functional but needs repairs');


In [0]:
#grab categorial features
X_train_cat= X_train.select_dtypes(exclude='number')
X_test_cat= X_test.select_dtypes(exclude='number')
test_features_cat= test_features.select_dtypes(exclude='number')

In [0]:
columns= ['recorded_by', 'extraction_type_group', 'extraction_type_class', 'management', 'quantity_group', 'quality_group', 'source_class', 'waterpoint_type_group', 'payment', 'source_type','wpt_name', 'subvillage', 'scheme_name', 'ward', 'installer', 'funder', 'date_recorded']
#drop columns i dont need
X_train_cat= X_train_cat.drop(columns= columns)
X_test_cat= X_test_cat.drop(columns= columns)
test_features_cat= test_features.drop(columns= columns)

In [14]:
X_train_cat.describe().T.sort_values(by='unique')

,count,unique,top,freq
public_meeting,44862,2,True,40792
permit,45028,2,True,31046
management_group,47520,5,user-group,42005
quantity,47520,5,enough,26586
payment_type,47520,7,never pay,20289
waterpoint_type,47520,7,communal standpipe,22815
water_quality,47520,8,soft,40617
basin,47520,9,Lake Victoria,8172
source,47520,10,spring,13578
scheme_management,44399,11,VWC,29450


In [0]:
#use one hot encoding on categorical features that are left
encoder= ce.OneHotEncoder(use_cat_names=True)
X_train_encoded= encoder.fit_transform(X_train_cat)
X_test_encoded= encoder.transform(X_test_cat)
test_features_encoded= encoder.transform(test_features_cat)

In [16]:
X_train_encoded.shape, X_test_encoded.shape

((47520, 233), (11880, 233))

In [14]:
#model with one hot encoded categorical features
model= LogisticRegression(solver='lbfgs', multi_class= 'auto', max_iter=1000)
model.fit(X_train_encoded, y_train)
y_pred= model.predict(X_test_encoded)
accuracy_score(y_test, y_pred)

0.7459595959595959

In [0]:
#coef for encoded features
functional_coef= pd.Series(model.coef_[0], X_train_encoded.columns)
non_functional_coef= pd.Series(model.coef_[1], X_train_encoded.columns)
needs_repairs_coef= pd.Series(model.coef_[2], X_train_encoded.columns)

basin_Lake Tanganyika                          0.25
basin_Pangani                                 -0.33
basin_Wami / Ruvu                             -0.41
basin_Lake Nyasa                               0.27
basin_Internal                                -0.00
basin_Lake Victoria                            0.25
basin_Rufiji                                  -0.03
basin_Lake Rukwa                               0.12
basin_Ruvuma / Southern Coast                 -0.12
region_Rukwa                                  -0.29
region_Kilimanjaro                             0.19
region_Dar es Salaam                           0.74
region_Mbeya                                  -0.20
region_Arusha                                  0.26
region_Tabora                                 -0.09
region_Tanga                                   0.69
region_Mwanza                                 -0.28
region_Morogoro                                0.07
region_Shinyanga                              -0.34
region_Dodom

In [0]:
#combined catergorical and numeric features
X_train_final= pd.concat([X_train_encoded,X_train_numeric], axis=1)
X_test_final= pd.concat([X_test_encoded, X_test_numeric],axis=1)
test_features_final= pd.concat([test_features_encoded, test_features_numeric], axis=1)

In [0]:
#standardized the data
scaler= StandardScaler()
X_train_final_scaled= scaler.fit_transform(X_train_final)
X_test_final_scaled= scaler.transform(X_test_final)
test_features_scaled= scaler.transform(test_features_final)

In [23]:
X_train_final_scaled.shape, X_test_final_scaled.shape, test_features_scaled.shape

((47520, 241), (11880, 241), (14358, 241))

In [34]:
#model with numeric and catergorical features
model= LogisticRegression(solver='saga',multi_class='auto',max_iter=1000)
model.fit(X_train_final_scaled, y_train)
y_pred= model.predict(X_test_final_scaled)
accuracy_score(y_test, y_pred)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


0.7485690235690236

In [0]:
#predicting test_features
y_pred= model.predict(test_features_scaled)

In [0]:
#placing my predictions in sample sub
sample_sub['status_group']= y_pred

In [27]:
#checking out amounts
sample_sub['status_group'].value_counts()

functional                 9415
non functional             4704
functional needs repair     239
Name: status_group, dtype: int64

In [0]:
sample_sub.to_csv('sample_sub.csv', index=False)

In [0]:
files.download('sample_sub.csv')